<a href="https://colab.research.google.com/github/Benish-Lezdo/ai_learnings/blob/main/LLAMA3.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LLAMA with RAG

In [12]:
%pip install llama-index==0.10.18 llama-index-llms-groq==0.1.3 groq==0.4.2 llama-index-embeddings-huggingface==0.2.0

INFO: pip is looking at multiple versions of llama-parse to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-parse to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 861.9/861.9 kB 56.

In [13]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    ServiceContext,
    load_index_from_storage
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.groq import Groq

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [14]:
from google.colab import userdata
GROQ_API_KEY = userdata.get('grok_llama3.1')

In [15]:
reader = SimpleDirectoryReader(input_files=["./dataset1.pdf"])
documents = reader.load_data()

In [16]:
text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=200)
nodes = text_splitter.get_nodes_from_documents(documents, show_progress=True)

Parsing nodes:   0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [17]:
llm = Groq(model="llama-3.1-8b-instant", api_key=GROQ_API_KEY)

In [21]:
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)

In [22]:
vector_index = VectorStoreIndex.from_documents(documents, show_progress=True, service_context=service_context, node_parser=nodes)

Parsing nodes:   0%|          | 0/10 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/10 [00:00<?, ?it/s]

In [23]:
vector_index.storage_context.persist(persist_dir="./index")

In [24]:
storage_context = StorageContext.from_defaults(persist_dir="./index")

In [25]:
index = load_index_from_storage(storage_context, service_context=service_context)

In [26]:
query_engine = index.as_query_engine(service_context=service_context)

In [31]:
query = "What is the name and age of the patient"
resp = query_engine.query(query)
resp.response

'Mr Tan Ah Kow, 55 years old.'

In [33]:
query = "Where is the hospital located give the full address ?"
resp = query_engine.query(query)
resp.response

'1 Blackacre Hospital, Singapore 01010101'

In [ ]:
query = "What is the current employment status of the patient ?"
resp = query_engine.query